# Content Based Recommendation Sytem

[참고 사이트](https:// [clikck here](https://github.com/lsjsj92/recommender_system_with_Python/blob/master/002.%20recommender%20system%20basic%20with%20Python%20-%201%20content%20based%20filtering.ipynb))

In [1]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#  youtube data instances

In [2]:
# get data
df = pd.read_csv('/content/drive/Shareddrives/2021-1 SDS Team3 (전현진, 김현지, 유태원)/csv/total_merged_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2535 entries, 0 to 2534
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2535 non-null   object 
 1   categoryId     2535 non-null   int64  
 2   description    2312 non-null   object 
 3   publishedAt    2535 non-null   object 
 4   title          2535 non-null   object 
 5   thmbnails      2535 non-null   object 
 6   channelTitle   2535 non-null   object 
 7   duration       2535 non-null   object 
 8   caption        2535 non-null   bool   
 9   viewCount      2533 non-null   float64
 10  likeCount      2502 non-null   float64
 11  dislikeCount   2502 non-null   float64
 12  favoriteCount  2535 non-null   int64  
 13  commentCount   2468 non-null   float64
 14  subtitle       2535 non-null   object 
dtypes: bool(1), float64(4), int64(2), object(8)
memory usage: 279.9+ KB


In [3]:
df = df.dropna(axis = 0)
df.info() # 2225개 instances

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2225 entries, 0 to 2534
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2225 non-null   object 
 1   categoryId     2225 non-null   int64  
 2   description    2225 non-null   object 
 3   publishedAt    2225 non-null   object 
 4   title          2225 non-null   object 
 5   thmbnails      2225 non-null   object 
 6   channelTitle   2225 non-null   object 
 7   duration       2225 non-null   object 
 8   caption        2225 non-null   bool   
 9   viewCount      2225 non-null   float64
 10  likeCount      2225 non-null   float64
 11  dislikeCount   2225 non-null   float64
 12  favoriteCount  2225 non-null   int64  
 13  commentCount   2225 non-null   float64
 14  subtitle       2225 non-null   object 
dtypes: bool(1), float64(4), int64(2), object(8)
memory usage: 262.9+ KB


In [4]:
# 삭제된 행으로 인한 index 재정렬
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2225 non-null   object 
 1   categoryId     2225 non-null   int64  
 2   description    2225 non-null   object 
 3   publishedAt    2225 non-null   object 
 4   title          2225 non-null   object 
 5   thmbnails      2225 non-null   object 
 6   channelTitle   2225 non-null   object 
 7   duration       2225 non-null   object 
 8   caption        2225 non-null   bool   
 9   viewCount      2225 non-null   float64
 10  likeCount      2225 non-null   float64
 11  dislikeCount   2225 non-null   float64
 12  favoriteCount  2225 non-null   int64  
 13  commentCount   2225 non-null   float64
 14  subtitle       2225 non-null   object 
dtypes: bool(1), float64(4), int64(2), object(8)
memory usage: 245.7+ KB


# 사용 columns
- id : 비디오 id
- description: 설명
- title : 제목
- likeCount: 좋아요 수
- viewCount: 조회수
- commentCount: 댓글 수

In [5]:
# 사용 데이터 뽑기
df = df[[ 'id', 'title', 'viewCount', 'commentCount', 'likeCount', 'description' ]]

df.head()

,id,title,viewCount,commentCount,likeCount,description
0,sM1Iwt7Lfrw,L6. Properties of Source Entropy,65.0,0.0,2.0,Entropy for a source is defined as the average...
1,FaZzm2bNdP0,ICM2014 VideoSeries IL14.2: Mark Braverman on ...,596.0,0.0,2.0,The International Congress of Mathematicians (...
2,3prNvXdATgg,Theory of Computation CS6160 Lecture 14 part 2...,46.0,0.0,1.0,This lecture is part of a course on the Theory...
3,NjiwlBgwfYg,Theory of Computation: Universality Theorem (P...,567.0,0.0,2.0,1) Statement of Universality Theorem\n2) Proof...
4,NQexlSJHeuw,NFA to DFA Conversion Example 1 | Lecture 26 |...,2485.0,5.0,53.0,Playlist Link\nTheory of computation Bangla Tu...


# content_based filtering by 'title'

In [6]:
# title로  content based_filtering한다. 
df.title.head()
#dtype : object --> 문자열을 벡터화시킴.

0                     L6. Properties of Source Entropy
1    ICM2014 VideoSeries IL14.2: Mark Braverman on ...
2    Theory of Computation CS6160 Lecture 14 part 2...
3    Theory of Computation: Universality Theorem (P...
4    NFA to DFA Conversion Example 1 | Lecture 26 |...
Name: title, dtype: object

In [7]:
count_vector = CountVectorizer(ngram_range=(1,3))
c_vector_dscp = count_vector.fit_transform(df['title'])
c_vector_dscp.shape

(2225, 18645)

In [8]:
#코사인 유사도를 구한 벡터를 미리 저장
dscp_c_sim = cosine_similarity(c_vector_dscp, c_vector_dscp).argsort()[:, ::-1]
dscp_c_sim.shape

(2225, 2225)

In [9]:
def get_recommend_video_list(df, video_id, top=30): # 유사한 최대 top 30개 뽑아냄.

    #video_id로 video_title 찾기.
    target_idx = df[df['id'] ==video_id].index.values[0]
    video_title = df.at[target_idx, 'title']
  
    # 특정 video와 비슷한 video를 추천해야 하기 때문에 'video' 정보를 뽑아낸다.
    target_video_index = df[df['title'] ==video_title].index.values
    # print(target_video_index)

    #코사인 유사도 중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = dscp_c_sim[target_video_index, :top].reshape(-1)
    
    
    #본인을 제외
    sim_index = sim_index[sim_index != target_video_index][:10]
    # print("sim index: {}".format(sim_index))

    #data frame으로 만들고 likeCount으로 정렬한 뒤 return
    result = df.iloc[sim_index] # 유사한 상위 10개의 video 뽑아냄.
    return list(result['id']) # video_id만 출력


In [10]:
videoID_lst = []
for i in range(2225):
  videoID_lst.append(df.loc[i,'id'])

# 전체 2225개의 영상별로 추천 영상 10개씩 출력 후 리스트에 저장

In [11]:
# 추천 video 목록 video별로 dictionary에 저장.
recommend_lst = []
error_video = []
for i in range(len(videoID_lst)):
  video_id = videoID_lst[i]
  # print(title)
  try:
    result = get_recommend_video_list(df, video_id=video_id)
    recommend_lst.append(result)
  except:
    error_video.append(video_id)
    continue

#print(recommend_lst)
print(error_video)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  app.launch_new_instance()


['GIov6vtG8OM', 'p23tzql2sag', 'r1EKIDpmNtU', 'Zh85gS4rgz0', 'PYn4Ldd_74g', 'Ev0Zd_RyVLw', 'bGwxpJ17JrE', 'C5ZgVM7jMOk', '89fKiaMxHrA', 'aHRGTSL4T4M', '8TM_FpcZuXA', '_1Acaj5aUwY', 'kYisRt51ALI', 'M6hxSujClEM', 'qdcBcOSRzic', 'nu3euYS0Dcs', '2b3uuwnMTw0', 'b3UAtxS8DWQ', 'yXqaclR6YaQ', 'Vy5xBsv6zYo', 'v2-G1AT6A-4', 'SRTSVxUnsNI', 'K8c5rnCUXRU', 'h8EijwzPths', 'dJh1MPiGka8', 'SbMBv8nU5pE', '_h7Nr6--gUQ', 'wrSDo2yKs00', 'SRTSVxUnsNI', 'fWkLMRcVTBM', 'vPHrYKTakbM', 'YwovapXfAVI', 'yReIQWSnwys', 'EAugbVPv68g', '89fKiaMxHrA', 'PS6UHoVmOC8']


# error idex를 가진 video_ID list 에서 해당 video_id 제거하기.

In [12]:
for i in error_video:
  videoID_lst.remove(i)
print(len(videoID_lst)) # 2189

2189


# videoID별 추천 영상 10개를 dictionary에 저장

In [13]:
# 최종적으로 dictionary에 저장.
new_dict  = {}
for i in range(len(videoID_lst)):
  new_dict[videoID_lst[i]] = recommend_lst[i]

print(new_dict)

{'sM1Iwt7Lfrw': ['M3UkYhwZImI', 'ibDa05SJFNg', 'FzkF5Qs5mYk', 'whJio_5kehM', '8RDq_7d5zcw', 'HKogW6P27WE', 'd5wvFcIzUwk', '-0zYV5YlaiM', 'ECNBf3KfOYM', 'RJtCR3h9mXQ'], 'FaZzm2bNdP0': ['0S08mv-YMng', '6aRJQNYYz4s', '_EopCPqpxzE', 'f1W_LMiQw9Y', '-7iCuKu9VF8', '_9qyD8Zo-JE', 'GE-XBf-Bchc', 'dwJIxsN852k', 'Nhqx5Ov5T8s', '6-YzxD5__0Y'], '3prNvXdATgg': ['gAdOEorTiG8', 'yocj0yc1C74', '7dJMaUBneAE', '_vVV7zrGgt0', '-YPpm0jCkFk', 'u5nOWjnf1so', 'PYn4Ldd_74g', 'Zh85gS4rgz0', 'dV-1afamttw', '8gaj42HICmk'], 'NjiwlBgwfYg': ['s00WotcO7W8', 'USY1z655tAs', '73ysiT68qeg', 'omq32WTbQys', 'qYRStlXI_Yo', '58N2N7zJGrQ', 'AfYbj6BCLQE', '8RDq_7d5zcw', 'Y8c_3LzmF-g', 'SDNZWWjJ23I'], 'NQexlSJHeuw': ['vp4T4EKgICk', 'hokaTjEIhRk', 'AI7TJEpZk84', 'etTlpznwKuU', 'kd79lAhC9X8', '1RF4B4V3Nyc', 'dxVGv19Mrg0', 'wROVFoNrlOA', 'SXrXIJ3mA8Q', 'EEbaxlxRWLQ'], 'LEtSguhw6JQ': ['qJsp7FamvhY', 'Zh85gS4rgz0', 'PYn4Ldd_74g', '80no_nZTKE4', 'aJK87QhAOjk', 'AG1nprrbWeg', 'li0lfERavcw', 'ym1qFgNkPqQ', 'Ncfuq9XAPus', 'ytmJ3lXZs44'